In [ ]:
import requests

url='https://dapi.kakao.com/v2/local/search/address.json'
headers={'Authorization':'KakaoAK c24cc35c9a9994cf2b9d881637ce0c19'}
params={'query':'서울특별시 중구 세종대로 110'}

resp=requests.get(url,headers=headers,params=params)
resp.json()

{'documents': [{'address': {'address_name': '서울 중구 태평로1가 31',
    'b_code': '1114010300',
    'h_code': '1114055000',
    'main_address_no': '31',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '중구',
    'region_3depth_h_name': '명동',
    'region_3depth_name': '태평로1가',
    'sub_address_no': '',
    'x': '126.977918351844',
    'y': '37.566370776634'},
   'address_name': '서울 중구 세종대로 110',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 중구 세종대로 110',
    'building_name': '서울특별시청',
    'main_building_no': '110',
    'region_1depth_name': '서울',
    'region_2depth_name': '중구',
    'region_3depth_name': '태평로1가',
    'road_name': '세종대로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.977918351844',
    'y': '37.566370776634',
    'zone_no': '04524'},
   'x': '126.977918351844',
   'y': '37.566370776634'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [ ]:
!pip install folium

import requests
import folium

KAKAO_REST_API_KEY='c24cc35c9a9994cf2b9d881637ce0c19'

In [ ]:
loc=[37.54,127.05]
map=folium.Map(location=loc)
map

In [ ]:
def geocode_kakao(address):
  url='https://dapi.kakao.com/v2/local/search/address.json'
  headers={'Authorization':'KakaoAK c24cc35c9a9994cf2b9d881637ce0c19'}
  params={'query':address}

  resp=requests.get(url,headers=headers,params=params)
  resp.raise_for_status()
  data=resp.json()
  if len(data['documents'])==0:
    raise ValueError("검색 결과가 없습니다. 주소를 다시 확인해주세요.")

  doc=data['documents'][0]
  lat=float(doc['y'])
  lng=float(doc['x'])

  return[lat,lng]


In [ ]:
addr="전북특별자치도 전주시 덕진구 백제대로 567"

loc=geocode_kakao(addr)
print("좌표:",loc)

m2=folium.Map(location=loc,zoom_start=16)
m2


좌표: [35.8411382714422, 127.131588969832]


In [ ]:
import folium
import requests

KAKAO_REST_API_KEY='c24cc35c9a9994cf2b9d881637ce0c19'

addr="서울 종로구 사직로 161"
loc=geocode_kakao(addr)
loc

[37.5759040910202, 126.976842133821]

In [ ]:
m=folium.Map(location=loc,zoom_start=16,tiles='OpenStreetMap')

folium.Marker(location=loc,popup='경복궁').add_to(m)
m

In [ ]:
import folium
import requests

KAKAO_REST_API_KEY='c24cc35c9a9994cf2b9d881637ce0c19'

addr="강원특별자치도 강릉시 창해로 514"
loc=geocode_kakao(addr)
loc

[37.8057701120176, 128.907481194757]

In [ ]:
m=folium.Map(location=loc,zoom_start=13)

folium.Marker(location=loc, icon=folium.Icon(color='red',icon='star')).add_to(m)
m

In [ ]:
html_start='<div style="\
                 font-size:12px;\
                 color:blue;\
                 background-color:rgba(255,255,255,0.2);\
                 width:85px;\
                 text-align;\
                 margin:0px;\
                 "><b>'

html_end='</b></div>'

In [ ]:
folium.Marker(location=loc, icon=folium.DivIcon(
    icon_anchor=(0,0),
    html=html_start+'경포해수욕장'+html_end
    )).add_to(m)

m

In [ ]:
from IPython.utils.text import list_strings
def geocode_kakao(address):
  url='https://dapi.kakao.com/v2/local/search/address.json'
  headers={'Authorization':'KakaoAK c24cc35c9a9994cf2b9d881637ce0c19'}
  params={'query':address}

  resp=requests.get(url,headers=headers,params=params)
  resp.raise_for_status()
  data=resp.json()
  if len(data['documents'])==0:
    raise ValueError("검색 결과가 없습니다. 주소를 다시 확인해주세요.")

  doc=data['documents'][0]
  lat=float(doc['y'])
  lng=float(doc['x'])

  return[lat,lng]

def add_coordinates_to_dataframe(df,addr_col):
  lats=[]
  lngs=[]
  for addr in df[addr_col]:
    lat,lng=geocode_kakao(addr)
    lats.append(lat)
    lngs.append(lng)
  df["decimalLatitude"]=lats
  df['decimalLongtitude']=lngs
  return df

In [ ]:
import pandas as pd
names=['용두암','성산일출봉','정방폭포','중문관광단지','한라산1100고지','차귀도']
addr=['제주시 용두암길 15',
      '서귀포시 성산읍 성산리',
      '서귀포시 동홍동 299-3',
      '서귀포시 중문동 2624-1',
      '서귀포시 색달동 산1-2',
      '제주시 한경면 고산리 산 117']

df=pd.DataFrame({'names':names,'addr':addr})
df=add_coordinates_to_dataframe(df,'addr')
df.dtypes

,0
names,object
addr,object
decimalLatitude,float64
decimalLongtitude,float64


In [ ]:
center=[df["decimalLatitude"].mean(),
        df["decimalLongtitude"].mean()]
center

[np.float64(33.35646904072083), np.float64(126.50771527972067)]

In [ ]:
map=folium.Map(location=center,zoom_start=10)

for i in range(len(df)):
  lat=df.loc[i,'decimalLatitude']
  lng=df.loc[i,'decimalLongtitude']
  name=df.loc[i,'names']

  folium.Marker(
      location=[lat,lng],
      icon=folium.Icon(color='red',icon='star'),
      popup=name).add_to(map)

In [ ]:
map

In [ ]:
html_start='<div style="\
                 font-size:12px;\
                 color:blue;\
                 background-color:rgba(255,255,255,0.2);\
                 width:85px;\
                 text-align;\
                 margin:0px;\
                 "><b>'

html_end='</b></div>'

for i in range(len(df)):
  folium.Marker(
      location=[df.iloc[i,2],df.iloc[i,3]],
      icon=folium.DivIcon(
          icon_anchor=(0,0),
          html=html_start+df.names[i]+html_end
      )
  ).add_to(map)

In [ ]:
map

In [ ]:
import folium
import pandas as pd
df=pd.read_csv('wind.csv')
df.head()

,lon,lat,dir,dur,std,spd
0,-99.900,24.3,3.141593,0.0,0.0,0.0
1,-99.856,24.3,3.141593,0.0,0.0,0.0
2,-99.811,24.3,3.141593,0.0,0.0,0.0
3,-99.767,24.3,3.141593,0.0,0.0,0.0
4,-99.723,24.3,3.141593,0.0,0.0,0.0


In [ ]:
df=df.sample(50,random_state=123)
center=[df.lat.mean(), df.lon.mean()]

map=folium.Map(location=center,zoom_start=5)
map

In [ ]:
for i in range(len(df)):
  folium.Marker(location=[df.lat.iloc[i],df.lon.iloc[i]],
                icon=folium.Icon(color='blue',icon='flag')).add_to(map)
map

In [ ]:
map=folium.Map(location=center,zoom_start=5)

for i in range(len(df)):
  folium.CircleMarker(location=[df.lat.iloc[i],df.lon.iloc[i]],
                      radius=(df.spd.iloc[i]**0.5)*2,
                      color='red',
                      stroke=False,
                      fill=True,
                      fill_opacity='50%').add_to(map)

In [ ]:
map